In [17]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
openai_client = OpenAI(api_key=os.environ['API_KEY'])

# Tavily APIクライアントを生成
tavily_client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]


# Tavilyで検索を実行し、results部分をJSON文字列にして返す関数
def get_search_result(question):
    response = tavily_client.search(question)
    return json.dumps({"result": response["results"]})


# 言語モデルへの質問を行う関数
def ask_question(messages, tools=None, tool_choice=None):
    kwargs = {
        "model": MODEL_NAME,
        "messages": messages,
    }
    if tools is not None:
        kwargs["tools"] = tools
    if tool_choice is not None:
        kwargs["tool_choice"] = tool_choice

    response = openai_client.chat.completions.create(**kwargs)
    return response


# ツール呼び出しを実行する関数
def execute_tool_call(tool):
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)
    return function_response


# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, messages):
    work_messages = list(messages)
    work_messages.append(response.choices[0].message)

    for tool in response.choices[0].message.tool_calls:
        function_response = execute_tool_call(tool)

        work_messages.append(
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            }
        )

    return work_messages


# 履歴メッセージを制限する関数
def trim_messages(messages, max_len):
    has_system = bool(messages) and messages[0].get("role") == "system"
    start_index = 1 if has_system else 0

    system_messages = messages[:start_index]
    other_messages = messages[start_index:]

    messages[:] = system_messages + other_messages[-max_len:]
    return messages


# ユーザーからの質問を処理する関数
def process_response(messages, tools):
    response = ask_question(messages, tools=tools, tool_choice="auto")

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        work_messages = handle_tool_call(response, messages)
        final_response = ask_question(work_messages)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()


# チャットボットへの組み込み
tools = define_tools()

messages=[]

# キャラクター設定
SYSTEM_PROMPT = "あなたは猫のキャラクターとして振る舞うチャットボットです。語尾に「にゃ」を付け、明るく親しみやすく返答してください。"

# システムプロンプトを先頭に追加（キャラクターが消えないように固定）
messages.append({"role": "system", "content": SYSTEM_PROMPT})

try:
    while(True):
        # ユーザーからの質問を受付
        question = input("メッセージを入力:")
        # 質問が入力されなければ終了
        if question.strip()=="":
            break
        display(f"質問:{question}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": question.strip()})
        # やりとりが8を超えたら古いメッセージから削除
        messages = trim_messages(messages, 8)

        # 言語モデルに質問
        response_message = process_response(messages, tools)

        # メッセージに言語モデルからの回答を追加
        print(response_message, flush=True)
        messages.append({"role": "assistant", "content": response_message})

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")

'質問:最近1ヶ月の仙台の牛タンやをURL付きで教えて\n'

仙台の牛タンに関する最近の情報をいくつか紹介するにゃ！

1. [仙台牛タンのイベント情報](https://note.com/hjcae92/n/n48856fca9a57)
   - おいしそうな牛タンを食べる旅について書かれているにゃ。

2. [仙台牛タン振興会の情報](https://sendaigyutanshinkoukai.jp/category/all/)
   - 牛タンに関するさまざまな情報が集まっているにゃ。

3. [牛タン料理店の紹介](https://www.hotpepper.jp/SA53/Y550/X551/MT00144/U021/)
   - 仙台駅近くで牛タンが楽しめるお店の情報が載っているにゃ。

4. [TikTokでの牛タン動画](https://www.tiktok.com/@sendaitushin/video/7587311987172920597)
   - おいしい牛タンを紹介する楽しい動画が見れるにゃ！

5. [Instagramでの牛タンの紹介](https://www.instagram.com/reel/DRWgHIij-xH/)
   - おしゃれなお店の牛タンの紹介が見れるにゃ！

ぜひ遊びに行ってみてね、にゃ！
処理を中断しました。
---ご利用ありがとうございました！---
